In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Esophageal_Cancer/GSE104958'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"A 17-molecule set as a predictor of complete response to neoadjuvant chemotherapy with docetaxel, cisplatin, and 5-fluorouracil in esophageal cancer"
!Series_summary	"Background  Recently, neoadjuvant chemotherapy with docetaxel/cisplatin/5-fluorouracil (NAC-DCF) was identified as a novel strong regimen with a high rate of pathological complete response (pCR) in advanced esophageal cancer in Japan. Predicting pCR will contribute to the therapeutic strategy and the prevention of surgical invasion. However, a predictor of pCR after NAC-DCF has not yet been developed. The aim of this study was to identify a novel predictor of pCR in locally advanced esophageal cancer treated with NAC-DCF.  Patients and Methods   A total of 32 patients who received NAC-DCF followed by esophagectomy between June 2013 and March 2016 were enrolled in this study. We divided the patients into the following 2 groups: pCR group (9 cases) and non-pCR group (23 cases), and com

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Check for gene expression data availability
# By looking at the background information and study design, it can be determined that the data is indeed gene expression data.
is_gene_available = True

# Define the keys for the available variables
trait_row = 1  # Assuming pCR status inferred here
age_row = None  # Age data is not explicitly found - re-check if possible
gender_row = None  # Gender data is not explicitly found - re-check if possible

# Function to convert 'Esophageal_Cancer' status (binary): pCR case (True) and non-pCR case (False)
def convert_trait(value):
    label = value.split(':')[-1].strip()
    if label == "pCR group":
        return 1
    elif label == "non-pCR group":
        return 0
    return None

# Dummy converter functions for age and gender (since they are unavailable)
def convert_age(value):
    return None

def convert_gender(value):
    return None

save_cohort_info('GSE104958', './preprocessed/Esophageal_Cancer/cohort_info.json', is_gene_available, trait_row is not None)

if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Esophageal_Cancer', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Esophageal_Cancer/trait_data/GSE104958.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM2811122': [None], 'GSM2811123': [None], 'GSM2811124': [None], 'GSM2811125': [None], 'GSM2811126': [None], 'GSM2811127': [None], 'GSM2811128': [None], 'GSM2811129': [None], 'GSM2811130': [None], 'GSM2811131': [None], 'GSM2811132': [None], 'GSM2811133': [None], 'GSM2811134': [None], 'GSM2811135': [None], 'GSM2811136': [None], 'GSM2811137': [None], 'GSM2811138': [None], 'GSM2811139': [None], 'GSM2811140': [None], 'GSM2811141': [None], 'GSM2811142': [None], 'GSM2811143': [None], 'GSM2811144': [None], 'GSM2811145': [None], 'GSM2811146': [None], 'GSM2811147': [None], 'GSM2811148': [None], 'GSM2811149': [None], 'GSM2811150': [None], 'GSM2811151': [None], 'GSM2811152': [None], 'GSM2811153': [None], 'GSM2811154': [None], 'GSM2811155': [None], 'GSM2811156': [None], 'GSM2811157': [None], 'GSM2811158': [None], 'GSM2811159': [None], 'GSM2811160': [None], 'GSM2811161': [None], 'GSM2811162': [None], 'GSM2811163': [None], 'GSM2811164': [None], 'GSM2811165': [None], 'GSM2811166': [None], 'GSM28111

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['(+)E1A_r60_1', '(+)E1A_r60_3', '(+)E1A_r60_a104', '(+)E1A_r60_a107',
       '(+)E1A_r60_a135', '(+)E1A_r60_a20', '(+)E1A_r60_a22', '(+)E1A_r60_a97',
       '(+)E1A_r60_n11', '(+)E1A_r60_n9', '3xSLv1', 'A_19_P00315452',
       'A_19_P00315492', 'A_19_P00315493', 'A_19_P00315502', 'A_19_P00315506',
       'A_19_P00315518', 'A_19_P00315519', 'A_19_P00315529', 'A_19_P00315541'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['GE_BrightCorner', 'DarkCorner', 'A_21_P0014386', 'A_33_P3396872', 'A_33_P3267760'], 'CONTROL_TYPE': ['pos', 'pos', 'FALSE', 'FALSE', 'FALSE'], 'REFSEQ': [nan, nan, nan, 'NM_001105533', nan], 'GB_ACC': [nan, nan, nan, 'NM_001105533', nan], 'LOCUSLINK_ID': [nan, nan, nan, 79974.0, 54880.0], 'GENE_SYMBOL': [nan, nan, nan, 'CPED1', 'BCOR'], 'GENE_NAME': [nan, nan, nan, 'cadherin-like and PC-esterase domain containing 1', 'BCL6 corepressor'], 'UNIGENE_ID': [nan, nan, nan, 'Hs.189652', nan], 'ENSEMBL_ID': [nan, nan, nan, nan, 'ENST00000378463'], 'ACCESSION_STRING': [nan, nan, nan, 'ref|NM_001105533|gb|AK025639|gb|BC030538|tc|THC2601673', 'ens|ENST00000378463'], 'CHROMOSOMAL_LOCATION': [nan, nan, 'unmapped', 'chr7:120901888-120901947', 'chrX:39909128-39909069'], 'CYTOBAND': [nan, nan, nan, 'hs|7q31.31', 'hs|Xp11.4'], 'DESCRIPTION': [nan, nan, nan, 'Homo sapiens cadherin-like and PC-esterase domain containing 1 (CPED1), transcript variant 2, mRNA [NM_001105533

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Determine the keys in the gene annotation dictionary
identifier_key = 'ID'
gene_symbol_key = 'GENE_SYMBOL'

# 2. Get the dataframe storing the mapping between probe IDs and genes
mapping_df = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Apply the mapping to convert the genetic data into gene expression data
gene_data = apply_gene_mapping(gene_data, mapping_df)


### Step 7: Data Normalization and Merging

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Esophageal_Cancer/gene_data/GSE104958.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Esophageal_Cancer')

# If the trait is not severely biased, save the cohort information and the merged data.


# 4. Save the cohort information.
save_cohort_info('GSE104958', './preprocessed/Esophageal_Cancer/cohort_info.json', True, True, trait_biased, unbiased_merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Esophageal_Cancer/GSE104958.csv'
    unbiased_merged_data.to_csv(csv_path)


Quartiles for 'Esophageal_Cancer':
  25%: nan
  50% (Median): nan
  75%: nan
Min: nan
Max: nan
The distribution of the feature 'Esophageal_Cancer' in this dataset is fine.

